<a href="https://colab.research.google.com/github/arunraja-hub/Preference_Extraction/blob/master/export_lucid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Install and imports

In [1]:
%tensorflow_version 1.x

!pip uninstall lucid -y
!pip install git+https://github.com/tensorflow/lucid.git#egg=lucid

!git clone https://github.com/arunraja-hub/Preference_Extraction.git

!pip install tf-agents==0.3.0

!pip uninstall tensorflow-probability -y
!pip install tensorflow-probability==0.7.0

!npm install -g svelte-cli@2.2.0

import tensorflow as tf

tf.compat.v1.disable_eager_execution()

TensorFlow 1.x selected.
Uninstalling lucid-0.3.8:
  Successfully uninstalled lucid-0.3.8
  Cloning https://github.com/tensorflow/lucid.git to /tmp/pip-install-4czqr4ke/lucid
  Running command git clone -q https://github.com/tensorflow/lucid.git /tmp/pip-install-4czqr4ke/lucid
  Created wheel for lucid: filename=lucid-0.3.9-cp36-none-any.whl size=155742 sha256=22948157518f6eb5cca797b231bac996cbf935188cc113aa416cbee0ca4ecc4a
  Stored in directory: /tmp/pip-ephem-wheel-cache-u3y8ii2j/wheels/64/c2/51/7238e75f457453774587708b21cd3cf32f9a86456648688634
Successfully built lucid
Cloning into 'Preference_Extraction'...
remote: Enumerating objects: 571, done.
remote: Counting objects: 100% (571/571), done.
remote: Compressing objects: 100% (557/557), done.
remote: Total 571 (delta 38), reused 523 (delta 13), pack-reused 0
Receiving objects: 100% (571/571), 20.89 MiB | 15.19 MiB/s, done.
Resolving deltas: 100% (38/38), done.
     |████████████████████████████████| 839kB 9.2MB/s 
     |██████████

In [0]:
import numpy as np

from tf_agents.trajectories.time_step import TimeStep
from tf_agents.specs.tensor_spec import TensorSpec
from tf_agents.specs.tensor_spec import TensorSpec
from tf_agents.specs.tensor_spec import BoundedTensorSpec
from tf_agents.networks import q_network

import lucid.modelzoo.vision_models as models

import concurrent.futures
import itertools
import os
import pickle
import random
import sys
import time

import numpy as np
import io
import collections

import urllib.request
from urllib.error import HTTPError

# Examine the checkpoint vars

In [0]:
cpt_name = "Preference_Extraction/model_ckpt"

In [4]:
cpt_var_names = tf.compat.v1.train.list_variables(cpt_name)
[name for name in cpt_var_names if (("bias" in name[0]) or ("kernel" in name[0])) and not ("OPTIMIZER" in name[0])]

[('agent/_q_network/_encoder/_postprocessing_layers/0/bias/.ATTRIBUTES/VARIABLE_VALUE',
  [16]),
 ('agent/_q_network/_encoder/_postprocessing_layers/0/kernel/.ATTRIBUTES/VARIABLE_VALUE',
  [3, 3, 5, 16]),
 ('agent/_q_network/_encoder/_postprocessing_layers/1/bias/.ATTRIBUTES/VARIABLE_VALUE',
  [32]),
 ('agent/_q_network/_encoder/_postprocessing_layers/1/kernel/.ATTRIBUTES/VARIABLE_VALUE',
  [3, 3, 16, 32]),
 ('agent/_q_network/_encoder/_postprocessing_layers/3/bias/.ATTRIBUTES/VARIABLE_VALUE',
  [64]),
 ('agent/_q_network/_encoder/_postprocessing_layers/3/kernel/.ATTRIBUTES/VARIABLE_VALUE',
  [960, 64]),
 ('agent/_q_network/_q_value_layer/bias/.ATTRIBUTES/VARIABLE_VALUE', [3]),
 ('agent/_q_network/_q_value_layer/kernel/.ATTRIBUTES/VARIABLE_VALUE',
  [64, 3]),
 ('agent/_target_q_network/_encoder/_postprocessing_layers/0/bias/.ATTRIBUTES/VARIABLE_VALUE',
  [16]),
 ('agent/_target_q_network/_encoder/_postprocessing_layers/0/kernel/.ATTRIBUTES/VARIABLE_VALUE',
  [3, 3, 5, 16]),
 ('agent/_t

# Setup model

In [5]:
tf.reset_default_graph()

input_shape = [14, 16, 5]

my_input = tf.placeholder(tf.float32, shape=[None] + input_shape, name="my_input")
q_vals = q_network.QNetwork(input_tensor_spec=TensorSpec(shape=(14, 16, 5)), action_spec=BoundedTensorSpec((), tf.int32, 0, 2), conv_layer_params = [[16, 3, 1], [32, 3, 2]], fc_layer_params = [64])(my_input)

[tensor for op in tf.get_default_graph().get_operations() for tensor in op.values()]

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


[<tf.Tensor 'my_input:0' shape=(?, 14, 16, 5) dtype=float32>,
 <tf.Tensor 'QNetwork/EncodingNetwork/EncodingNetwork/conv2d/kernel/Initializer/truncated_normal/shape:0' shape=(4,) dtype=int32>,
 <tf.Tensor 'QNetwork/EncodingNetwork/EncodingNetwork/conv2d/kernel/Initializer/truncated_normal/mean:0' shape=() dtype=float32>,
 <tf.Tensor 'QNetwork/EncodingNetwork/EncodingNetwork/conv2d/kernel/Initializer/truncated_normal/stddev:0' shape=() dtype=float32>,
 <tf.Tensor 'QNetwork/EncodingNetwork/EncodingNetwork/conv2d/kernel/Initializer/truncated_normal/TruncatedNormal:0' shape=(3, 3, 5, 16) dtype=float32>,
 <tf.Tensor 'QNetwork/EncodingNetwork/EncodingNetwork/conv2d/kernel/Initializer/truncated_normal/mul:0' shape=(3, 3, 5, 16) dtype=float32>,
 <tf.Tensor 'QNetwork/EncodingNetwork/EncodingNetwork/conv2d/kernel/Initializer/truncated_normal:0' shape=(3, 3, 5, 16) dtype=float32>,
 <tf.Tensor 'QNetwork/EncodingNetwork/EncodingNetwork/conv2d/kernel:0' shape=() dtype=resource>,
 <tf.Tensor 'QNetwor

# Get the var_name_to_prev_var_name mapping by matching tensors with the same shape.

In [6]:
cpt_var_info = tf.compat.v1.train.list_variables(cpt_name)
cpt_var_info = [var for var in cpt_var_info if (("bias" in var[0]) or ("kernel" in var[0])) and not ("OPTIMIZER" in var[0]) and not ("_target_q_network" in var[0])]

shape_to_cpt_var_name = {tuple(var[1]): var[0] for var in cpt_var_info}
shape_to_cpt_var_name

{(3,): 'agent/_q_network/_q_value_layer/bias/.ATTRIBUTES/VARIABLE_VALUE',
 (3,
  3,
  5,
  16): 'agent/_q_network/_encoder/_postprocessing_layers/0/kernel/.ATTRIBUTES/VARIABLE_VALUE',
 (3,
  3,
  16,
  32): 'agent/_q_network/_encoder/_postprocessing_layers/1/kernel/.ATTRIBUTES/VARIABLE_VALUE',
 (16,): 'agent/_q_network/_encoder/_postprocessing_layers/0/bias/.ATTRIBUTES/VARIABLE_VALUE',
 (32,): 'agent/_q_network/_encoder/_postprocessing_layers/1/bias/.ATTRIBUTES/VARIABLE_VALUE',
 (64,): 'agent/_q_network/_encoder/_postprocessing_layers/3/bias/.ATTRIBUTES/VARIABLE_VALUE',
 (64, 3): 'agent/_q_network/_q_value_layer/kernel/.ATTRIBUTES/VARIABLE_VALUE',
 (960,
  64): 'agent/_q_network/_encoder/_postprocessing_layers/3/kernel/.ATTRIBUTES/VARIABLE_VALUE'}

In [7]:
current_vars = tf.get_collection(tf.GraphKeys.VARIABLES)
current_vars

shape_to_current_var_name = {tuple(var.get_shape().as_list()): var.name[:-2] for var in current_vars}
shape_to_current_var_name

Instructions for updating:
Use `tf.GraphKeys.GLOBAL_VARIABLES` instead.


Instructions for updating:
Use `tf.GraphKeys.GLOBAL_VARIABLES` instead.


{(3,): 'QNetwork/dense/bias',
 (3, 3, 5, 16): 'QNetwork/EncodingNetwork/EncodingNetwork/conv2d/kernel',
 (3, 3, 16, 32): 'QNetwork/EncodingNetwork/EncodingNetwork/conv2d_1/kernel',
 (16,): 'QNetwork/EncodingNetwork/EncodingNetwork/conv2d/bias',
 (32,): 'QNetwork/EncodingNetwork/EncodingNetwork/conv2d_1/bias',
 (64,): 'QNetwork/EncodingNetwork/EncodingNetwork/dense/bias',
 (64, 3): 'QNetwork/dense/kernel',
 (960, 64): 'QNetwork/EncodingNetwork/EncodingNetwork/dense/kernel'}

In [8]:
var_name_to_prev_var_name = {}

for shape in shape_to_current_var_name:
  var_name_to_prev_var_name[shape_to_current_var_name[shape]] = shape_to_cpt_var_name[shape]

var_name_to_prev_var_name

{'QNetwork/EncodingNetwork/EncodingNetwork/conv2d/bias': 'agent/_q_network/_encoder/_postprocessing_layers/0/bias/.ATTRIBUTES/VARIABLE_VALUE',
 'QNetwork/EncodingNetwork/EncodingNetwork/conv2d/kernel': 'agent/_q_network/_encoder/_postprocessing_layers/0/kernel/.ATTRIBUTES/VARIABLE_VALUE',
 'QNetwork/EncodingNetwork/EncodingNetwork/conv2d_1/bias': 'agent/_q_network/_encoder/_postprocessing_layers/1/bias/.ATTRIBUTES/VARIABLE_VALUE',
 'QNetwork/EncodingNetwork/EncodingNetwork/conv2d_1/kernel': 'agent/_q_network/_encoder/_postprocessing_layers/1/kernel/.ATTRIBUTES/VARIABLE_VALUE',
 'QNetwork/EncodingNetwork/EncodingNetwork/dense/bias': 'agent/_q_network/_encoder/_postprocessing_layers/3/bias/.ATTRIBUTES/VARIABLE_VALUE',
 'QNetwork/EncodingNetwork/EncodingNetwork/dense/kernel': 'agent/_q_network/_encoder/_postprocessing_layers/3/kernel/.ATTRIBUTES/VARIABLE_VALUE',
 'QNetwork/dense/bias': 'agent/_q_network/_q_value_layer/bias/.ATTRIBUTES/VARIABLE_VALUE',
 'QNetwork/dense/kernel': 'agent/_q_n

# Read data

In [0]:
class Trajectory(
    collections.namedtuple('Trajectory', [
        'step_type',
        'observation',
        'action',
        'policy_info',
        'next_step_type',
        'reward',
        'discount',
    ])):
  """Stores the observation the agent saw and the action it took.
      The rest of the attributes aren't used in this code."""
  __slots__ = ()

class ListWrapper(object):
  def __init__(self, list_to_wrap):
    self._list = list_to_wrap

  def as_list(self):
    return self._list

class RenameUnpickler(pickle.Unpickler):
    def find_class(self, module, name):
      if name == "Trajectory":
        return Trajectory
      if name == "ListWrapper":
        return ListWrapper

      return super(RenameUnpickler, self).find_class(module, name)

def rename_load(s):
    """Helper function analogous to pickle.loads()."""
    return RenameUnpickler(s, encoding='latin1').load()

In [0]:
# Modified read trajectories functions to read files from local storage

def load_file(full_path):
    try:
        with open(full_path, 'rb') as f:
            data = rename_load(f)
            return data
    except:
        return None
    
def all_load_data(base_path):
    
    executor = concurrent.futures.ThreadPoolExecutor(max_workers=100)
    
    futures = []
    for i in range(5000):
        full_path = os.path.join(base_path, "ts"+str(i)+".pickle")
        future = executor.submit(load_file, full_path)
        futures.append(future)
    
    raw_data = []
    for future in concurrent.futures.as_completed(futures):
        result = future.result()
        if result:
            raw_data.append(result)
    
    return raw_data


all_raw_data = all_load_data("Preference_Extraction/data/simple_env_1/")

# Do the warmstart and verify it worked

In [11]:
tf.train.warm_start(cpt_name, var_name_to_prev_var_name=var_name_to_prev_var_name)
init_op = tf.global_variables_initializer()

INFO:tensorflow:Warm-starting from: Preference_Extraction/model_ckpt


INFO:tensorflow:Warm-starting from: Preference_Extraction/model_ckpt


INFO:tensorflow:Warm-starting variables only in TRAINABLE_VARIABLES.


INFO:tensorflow:Warm-starting variables only in TRAINABLE_VARIABLES.


INFO:tensorflow:Warm-started 8 variables.


INFO:tensorflow:Warm-started 8 variables.


In [14]:
activation_tensor = tf.get_default_graph().get_tensor_by_name("QNetwork/EncodingNetwork/EncodingNetwork/dense/Relu:0")
weights = tf.get_default_graph().get_tensor_by_name("QNetwork/dense/kernel/Read/ReadVariableOp:0")
bias = tf.get_default_graph().get_tensor_by_name("QNetwork/dense/bias/Read/ReadVariableOp:0")

with tf.Session() as sess:
  sess.run(init_op)

  for i in range(len(all_raw_data[0].observation)):
    single_observation = np.array([all_raw_data[0].observation[i]])

    restored_activations = sess.run(activation_tensor, {my_input: single_observation})[0]
    old_activations = all_raw_data[0].policy_info["activations"][i]

    if i < 3:
      print("restored_activations", restored_activations, "old_activations", old_activations)

    np.testing.assert_allclose(restored_activations, old_activations, rtol=.1)

restored_activations [  0.         0.         0.         0.       145.40762    0.
   0.        36.025948   0.         0.         0.        37.78504
   0.         0.         0.         0.         0.         0.
   0.         0.         0.       122.21044    0.         0.
   0.         0.         0.        21.95644    0.         0.
   0.         0.         0.         0.       159.66725    0.
  53.03794    0.         0.         0.         0.         0.
   0.         0.       136.03       0.         0.         0.
   0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.         0.         0.
   0.        73.01822    0.         0.      ] old_activations [  0.         0.         0.         0.       145.40771    0.
   0.        36.025852   0.         0.         0.        37.785103
   0.         0.         0.         0.         0.         0.
   0.         0.         0.       122.21045    0.         0.
   0.         0.         0.        21.956379   0.   

# Save Lucid model.

In [15]:
with tf.Session() as sess:
  sess.run(init_op)

  models.Model.suggest_save_args()
  
  models.Model.save(
    input_name='my_input',
    image_shape=input_shape,
    output_names=["QNetwork/dense/BiasAdd"],
    image_value_range=[0,1],
    save_url="lucid_save_model.pb"
  )

Inferred: input_name = my_input (because it was the only Placeholder in the graph_def)
Inferred: image_shape = [14, 16, 5]
# Please sanity check all inferred values before using this code.
# Incorrect `image_value_range` is the most common cause of feature visualization bugs! Most methods will fail silently with incorrect visualizations!
Model.save(
    input_name='my_input',
    image_shape=[14, 16, 5],
    output_names=_,                   # TODO (eg. ['logits'])
    image_value_range=_,                   # TODO (eg. '[-1, 1], [0, 1], [0, 255], or [-117, 138]')
  )



/usr/local/lib/python3.6/dist-packages/lucid/modelzoo/vision_base.py:283: UserWarning: Could not infer output_names.
  warnings.warn("Could not infer output_names.")



Instructions for updating:
Use `tf.compat.v1.graph_util.remove_training_nodes`


Instructions for updating:
Use `tf.compat.v1.graph_util.remove_training_nodes`


Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`


...

Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`


INFO:tensorflow:Froze 8 variables.


INFO:tensorflow:Froze 8 variables.


INFO:tensorflow:Converted 8 variables to const ops.


INFO:tensorflow:Converted 8 variables to const ops.
